In [1]:
#!/usr/bin/python3
# -*-coding:utf-8
'''
Created on Fri Dec 1 22:22:35 2017

@author: Ray

'''
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


In [2]:
##################################################
# Load  
##################################################
col = ['msno','plan_list_price','payment_plan_days','actual_amount_paid','payment_method_id','transaction_date','membership_expire_date']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions', col) # 20,000,000

transactions = transactions.head(n = 500)

input_col = ['msno','t-1_transaction_date','t-1_membership_expire_date','t-1_actual_amount_paid',
             'order_number','t-1_payment_method_id','t-1_plan_list_price','t-1_payment_plan_days']
tran_time_diff = utils.read_multiple_csv('../../input/preprocessed_data/transactions_time_diff', input_col)
tran_time_diff = tran_time_diff.dropna() # drop columns whose oder_number == 1

tran_time_diff = tran_time_diff.head(n = 500)

##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
tran_time_diff['t-1_membership_expire_date']  = tran_time_diff['t-1_membership_expire_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
tran_time_diff['t-1_transaction_date']  = tran_time_diff['t-1_transaction_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

gc.collect()


  0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

# how does user feel like kkbox

In [87]:
def days_since_the_last_expiration(x):
    # x: each row of dataframe, series
    x['days_since_the_last_expiration'] = [i.days for i in (x.transaction_date - x['t-1_membership_expire_date'])]
    return x
def days_since_the_last_subscription(x):
    x['days_since_the_last_subscription'] = [i.days for i in (x.transaction_date - x['t-1_transaction_date'])]
    return x
def is_subscribe_early(x):
    x['is_subscribe_early'] = [1 if i <0 else 0 for i in x.days_since_the_last_expiration]
    return x
def creat_loyalty_trend(x):
    # date
    x['days_since_the_last_expiration-cumsum'] = x.days_since_the_last_expiration.cumsum()
    x['days_since_the_last_expiration_ratio'] = x.days_since_the_last_expiration.cumsum()/ x.order_number
    x['days_since_the_last_subscription_ratio'] = x.days_since_the_last_subscription.cumsum()/ x.order_number
    x['days_since_the_last_expiration_diff'] = x.days_since_the_last_expiration - x.days_since_the_last_expiration.shift(1)
    x['days_since_the_first_subscription'] = x.days_since_the_last_subscription.cumsum()
    # payment_method
    x['do_change_payment_method'] = [1 if p_m != t_1_p_m else 0 for p_m, t_1_p_m in x[['payment_method_id','t-1_payment_method_id']].values]
    # plan_list_price(這次訂單,有選擇更高的價錢的方案麼)
    x['do_spend_more_money'] = [p_price - t_1_p_price for p_price, t_1_p_price in x[['plan_list_price','t-1_plan_list_price']].values]
    # payment_plan_days(這次訂單,有選擇天數更高的方案麼)
    x['do_extend_payment_days'] = [p_p_days - t_1_p_days for p_p_days, t_1_p_days in x[['payment_plan_days','t-1_payment_plan_days']].values]
    # (這次訂單,真實付的錢有比上次多麽)
    x['do_paid_more'] = [a_paid - a_paid for a_paid, a_paid in x[['actual_amount_paid','t-1_actual_amount_paid']].values]
    return x


In [90]:
#==============================================================================
# def
#==============================================================================
def make(T):
    """
    T = 0
    folder = 'trainW-0'
    """
    if T ==-1:
        folder = 'test'
        test = pd.read_csv('../../input/sample_submission_v2.csv')
    else:
        folder = 'trainW-'+str(T)
        train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))
    #merge data

    if T == 0:
        # w = 0:使用3月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-02-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-01-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        df = pd.merge(test, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-04-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del test
        df['w'] = T 
    gc.collect()

    df = pd.merge(df, tran_time_diff, on='msno', how='left').drop_duplicates(['msno','order_number'])   
    
    df = df.dropna()

    # creating features
    df = df.groupby('msno').apply(days_since_the_last_expiration)
    df = df.groupby('msno').apply(days_since_the_last_subscription)
    df = df.groupby('msno').apply(is_subscribe_early)
    df = df.groupby('msno').apply(creat_loyalty_trend)

    output_col = ['msno','w','days_since_the_last_expiration',
            'days_since_the_last_subscription','is_subscribe_early','order_number',
            'days_since_the_last_expiration-cumsum',
            'days_since_the_last_expiration_ratio',
            'days_since_the_last_subscription_ratio',
            'days_since_the_last_expiration_diff',
            'days_since_the_first_subscription',
                  'do_change_payment_method',
                  'do_spend_more_money',
                  'do_spend_more_money',
                  'do_extend_payment_days',
                  'do_paid_more',
            ]
    df = df[output_col] # msno is not unique
    gc.collect()
    print ('{0} done'.format(T))
    return df

In [91]:
df = make(0)
df

0 done


,msno,w,days_since_the_last_expiration,days_since_the_last_subscription,is_subscribe_early,order_number,days_since_the_last_expiration-cumsum,days_since_the_last_expiration_ratio,days_since_the_last_subscription_ratio,days_since_the_last_expiration_diff,days_since_the_first_subscription,do_change_payment_method,do_spend_more_money,do_spend_more_money,do_extend_payment_days,do_paid_more
58581,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-29,0,1,2.0,-29,-14.500000,0.000000,NaN,0,0,0.0,0.0,0.0,0.0
58582,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-60,-29,1,3.0,-89,-29.666667,-9.666667,-31.0,-29,0,0.0,0.0,0.0,0.0
58583,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-91,-60,1,4.0,-180,-45.000000,-22.250000,-31.0,-89,0,0.0,0.0,0.0,0.0
58584,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-121,-91,1,5.0,-301,-60.200000,-36.000000,-30.0,-180,0,0.0,0.0,0.0,0.0
58585,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-152,-121,1,6.0,-453,-75.500000,-50.166667,-31.0,-301,0,0.0,0.0,0.0,0.0
58586,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-182,-152,1,7.0,-635,-90.714286,-64.714286,-30.0,-453,0,0.0,0.0,0.0,0.0
58587,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-213,-182,1,8.0,-848,-106.000000,-79.375000,-31.0,-635,0,0.0,0.0,0.0,0.0
58588,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-244,-213,1,9.0,-1092,-121.333333,-94.222222,-31.0,-848,0,0.0,0.0,0.0,0.0
58589,++1Wu2wKBA60W9F9sMh15RXmh1wN1fjoVGzNqvw/Gro=,0,-272,-244,1,10.0,-1364,-136.400000,-109.200000,-28.0,-1092,0,0.0,0.0,0.0,0.0
86451,++4RuqBw0Ss6bQU4oMxaRlbBPoWzoEiIZaxPM04Y4+U=,0,-31,0,1,2.0,-31,-15.500000,0.000000,NaN,0,0,0.0,0.0,0.0,0.0


In [ ]:
x['do_change_payment_method'] = [1 if p_m != t_1_p_m else 0 for p_m, t_1_p_m in x[['payment_method_id','t-1_payment_method_id']].values]

In [ ]:
# is_auto_renew and is_cancel

In [4]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory


##################################################
# Load  
##################################################
col = ['msno','plan_list_price','payment_plan_days','actual_amount_paid','payment_method_id','transaction_date','membership_expire_date']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions', col) # 20,000,000

transactions = transactions.head(n = 500)

input_col = ['msno','t-1_transaction_date','t-1_membership_expire_date','t-1_actual_amount_paid',
             'order_number_rev','t-1_payment_method_id','t-1_plan_list_price','t-1_payment_plan_days']
tran_time_diff = utils.read_multiple_csv('../../input/preprocessed_data/transactions_time_diff', input_col)
tran_time_diff = tran_time_diff.dropna() # drop columns whose oder_number == 1

tran_time_diff = tran_time_diff.head(n = 500)

##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
tran_time_diff['t-1_membership_expire_date']  = tran_time_diff['t-1_membership_expire_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
tran_time_diff['t-1_transaction_date']  = tran_time_diff['t-1_transaction_date'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

gc.collect()

print ('After Loading ...')




100%|██████████| 4/4 [00:29<00:00,  7.35s/it]

100%|██████████| 4/4 [00:00<00:00, 298.15it/s]

After Loading ...


In [15]:
x = transactions[transactions.msno == '+++IZseRRiQS9aaSkH6cMYU6bGDcxUieAi/tH67sC5s=']

In [16]:
list(reversed(x.order_number_rev))

AttributeError: 'DataFrame' object has no attribute 'order_number_rev'

In [5]:
def days_since_the_last_expiration(x):
    # x: each row of dataframe, series
    x['days_since_the_last_expiration'] = [i.days for i in (x.transaction_date - x['t-1_membership_expire_date'])]
    return x
def days_since_the_last_subscription(x):
    x['days_since_the_last_subscription'] = [i.days for i in (x.transaction_date - x['t-1_transaction_date'])]
    return x
def is_subscribe_early(x):
    x['is_subscribe_early'] = [1 if i <0 else 0 for i in x.days_since_the_last_expiration]
    return x
def creat_loyalty_trend(x):
    # date
    x['days_since_the_last_expiration-cumsum'] = x.days_since_the_last_expiration.cumsum()
    x['days_since_the_last_expiration_ratio'] = x.days_since_the_last_expiration.cumsum()/ list(reversed(x.order_number_rev))
    x['days_since_the_last_subscription_ratio'] = x.days_since_the_last_subscription.cumsum()/ list(reversed(x.order_number_rev))
    x['days_since_the_last_expiration_diff'] = x.days_since_the_last_expiration - x.days_since_the_last_expiration.shift(1)
    x['days_since_the_first_subscription'] = x.days_since_the_last_subscription.cumsum()
    # payment_method
    x['do_change_payment_method'] = [1 if p_m != t_1_p_m else 0 for p_m, t_1_p_m in x[['payment_method_id','t-1_payment_method_id']].values]
    # plan_list_price(這次訂單,有選擇更高的價錢的方案麼)
    x['do_spend_more_money'] = [p_price - t_1_p_price for p_price, t_1_p_price in x[['plan_list_price','t-1_plan_list_price']].values]
    # payment_plan_days(這次訂單,有選擇天數更高的方案麼)
    x['do_extend_payment_days'] = [p_p_days - t_1_p_days for p_p_days, t_1_p_days in x[['payment_plan_days','t-1_payment_plan_days']].values]
    # (這次訂單,真實付的錢有比上次多麽)
    x['do_paid_more'] = [a_paid - a_paid for a_paid, a_paid in x[['actual_amount_paid','t-1_actual_amount_paid']].values]
    return x
days_since_the_last_transactions = []


In [10]:
#==============================================================================
# def
#==============================================================================
def make(T):
    """
    T = 0
    folder = 'trainW-0'
    """
    if T ==-1:
        folder = 'test'
        test = pd.read_csv('../../input/sample_submission_v2.csv')
    else:
        folder = 'trainW-'+str(T)
        train = pd.read_csv('../../input/preprocessed_data/trainW-{0}.csv'.format(T))
    #merge data

    if T == 0:
        # w = 0:使用3月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-03-01', '%Y-%m-%d'))], 
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == 1:
        # w = 1:使用2月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-02-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == 2:
        # w = 2:使用1月之前的資料當作history
        df = pd.merge(train, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-01-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del train
    elif T == -1:
        # w = -1:使用4月之前的資料當作history
        df = pd.merge(test, 
        transactions[(transactions.transaction_date < datetime.strptime('2017-04-01', '%Y-%m-%d'))],
        on='msno', 
        how='left') # 此時msno就是不重複
        del test
        df['w'] = T 
    gc.collect()

    df = pd.merge(df, tran_time_diff, on='msno', how='left').drop_duplicates(['msno','order_number_rev'])   
    
    df = df.dropna()

    # creating features
    df = df.groupby('msno').apply(days_since_the_last_expiration)
    df = df.groupby('msno').apply(days_since_the_last_subscription)
    df = df.groupby('msno').apply(is_subscribe_early)
    df = df.groupby('msno').apply(creat_loyalty_trend)

    output_col = ['msno','w','days_since_the_last_expiration',
            'days_since_the_last_subscription','is_subscribe_early','order_number_rev',
            'days_since_the_last_expiration-cumsum',
            'days_since_the_last_expiration_ratio',
            'days_since_the_last_subscription_ratio',
            'days_since_the_last_expiration_diff',
            'days_since_the_first_subscription',
                  'do_change_payment_method',
                  'do_spend_more_money',
                  'do_spend_more_money',
                  'do_extend_payment_days',
                  'do_paid_more',
            ]
    df = df[output_col] # msno is not unique
    gc.collect()

    days_since_the_last_transactions.append(df)
    print ('{0} done'.format(T))
    del df
    gc.collect()
# 優化措施(Speed)    
# 多花一些記憶題把它寫下來(只會用一次), 如果可以避免之後反覆I/O,這樣反而是省時間的,






In [11]:
##################################################
# Main
##################################################
make(0)


KeyError: 3